In [1]:
from flask import Flask, render_template, session, request, redirect, url_for, flash, jsonify
from flask_restful import reqparse, abort, Api, Resource
import datetime as dt
import numpy as np
import pandas as pd
import time
import datetime

from flask_cors import CORS

In [2]:
data = pd.read_csv("data_time_spent.csv", sep = ';', encoding = 'utf-8')

In [3]:
data.head()

,UNIT,GEO,TIME,DAYSWEEK,SEX,ACL00,Value
0,Time spent (hh:mm),Belgium,2000,All days of the week,Total,Total,24:00:00
1,Time spent (hh:mm),Belgium,2000,All days of the week,Total,Personal care,10:58
2,Time spent (hh:mm),Belgium,2000,All days of the week,Total,Sleep,08:25
3,Time spent (hh:mm),Belgium,2000,All days of the week,Total,Eating,01:49
4,Time spent (hh:mm),Belgium,2000,All days of the week,Total,Other and/or unspecified personal care,00:44


In [4]:
data = data.rename(columns={"GEO": "COUNTRY", "TIME": "YEAR", 'ACL00' : 'CATEGORY'})
data.head()

,UNIT,COUNTRY,YEAR,DAYSWEEK,SEX,CATEGORY,Value
0,Time spent (hh:mm),Belgium,2000,All days of the week,Total,Total,24:00:00
1,Time spent (hh:mm),Belgium,2000,All days of the week,Total,Personal care,10:58
2,Time spent (hh:mm),Belgium,2000,All days of the week,Total,Sleep,08:25
3,Time spent (hh:mm),Belgium,2000,All days of the week,Total,Eating,01:49
4,Time spent (hh:mm),Belgium,2000,All days of the week,Total,Other and/or unspecified personal care,00:44


In [5]:
data = data[data['CATEGORY'] != 'Total']

In [6]:
data.dtypes

UNIT        object
COUNTRY     object
YEAR         int64
DAYSWEEK    object
SEX         object
CATEGORY    object
Value       object
dtype: object

In [7]:
data.shape

(62370, 7)

In [8]:
data['UNIT'].unique()

array(['Time spent (hh:mm)', 'Participation time (hh:mm)',
       'Participation rate (%)'], dtype=object)

In [9]:
test = data.pivot(values='Value', columns='UNIT')

In [10]:
test.head()

UNIT,Participation rate (%),Participation time (hh:mm),Time spent (hh:mm)
1,None,None,10:58
2,None,None,08:25
3,None,None,01:49
4,None,None,00:44
5,None,None,02:29


In [11]:
test.shape

(62370, 3)

In [12]:
result = pd.concat([data, test], axis=1)

In [13]:
result.head()

,UNIT,COUNTRY,YEAR,DAYSWEEK,SEX,CATEGORY,Value,Participation rate (%),Participation time (hh:mm),Time spent (hh:mm)
1,Time spent (hh:mm),Belgium,2000,All days of the week,Total,Personal care,10:58,None,None,10:58
2,Time spent (hh:mm),Belgium,2000,All days of the week,Total,Sleep,08:25,None,None,08:25
3,Time spent (hh:mm),Belgium,2000,All days of the week,Total,Eating,01:49,None,None,01:49
4,Time spent (hh:mm),Belgium,2000,All days of the week,Total,Other and/or unspecified personal care,00:44,None,None,00:44
5,Time spent (hh:mm),Belgium,2000,All days of the week,Total,"Employment, related activities and travel as p...",02:29,None,None,02:29


In [14]:
result1 = result[result['UNIT'] == 'Time spent (hh:mm)']

In [15]:
result1 = result1.drop(['UNIT', 'Value', "Participation rate (%)", "Participation time (hh:mm)"], axis=1)
result1 = result1.reset_index().drop("index", axis = 1)
result1.head()

,COUNTRY,YEAR,DAYSWEEK,SEX,CATEGORY,Time spent (hh:mm)
0,Belgium,2000,All days of the week,Total,Personal care,10:58
1,Belgium,2000,All days of the week,Total,Sleep,08:25
2,Belgium,2000,All days of the week,Total,Eating,01:49
3,Belgium,2000,All days of the week,Total,Other and/or unspecified personal care,00:44
4,Belgium,2000,All days of the week,Total,"Employment, related activities and travel as p...",02:29


In [16]:
result1.shape

(20790, 6)

In [17]:
result2 = result[result['UNIT'] == 'Participation time (hh:mm)']

In [18]:
result2 = result2.drop(['UNIT', 'Value', 'Time spent (hh:mm)', "Participation rate (%)", "COUNTRY", 
                        'YEAR', "DAYSWEEK", "SEX", "CATEGORY"], axis=1)
result2 = result2.reset_index().drop("index", axis = 1)
result2.head()

,Participation time (hh:mm)
0,10:59
1,08:25
2,01:50
3,00:45
4,07:11


In [19]:
result2.shape

(20790, 1)

In [20]:
result3 = result[result['UNIT'] == 'Participation rate (%)']

In [21]:
result3 = result3.drop(['UNIT', 'Value', 'Time spent (hh:mm)', "Participation time (hh:mm)", "COUNTRY", 
                        'YEAR', "DAYSWEEK", "SEX", "CATEGORY"], axis=1)
result3 = result3.reset_index().drop("index", axis = 1)
result3.head()

,Participation rate (%)
0,100.0
1,100.0
2,99.5
3,97.6
4,34.5


In [22]:
result3.shape

(20790, 1)

In [23]:
final_result = pd.concat([result1, result2, result3], axis=1)

In [24]:
final_result.head()

,COUNTRY,YEAR,DAYSWEEK,SEX,CATEGORY,Time spent (hh:mm),Participation time (hh:mm),Participation rate (%)
0,Belgium,2000,All days of the week,Total,Personal care,10:58,10:59,100.0
1,Belgium,2000,All days of the week,Total,Sleep,08:25,08:25,100.0
2,Belgium,2000,All days of the week,Total,Eating,01:49,01:50,99.5
3,Belgium,2000,All days of the week,Total,Other and/or unspecified personal care,00:44,00:45,97.6
4,Belgium,2000,All days of the week,Total,"Employment, related activities and travel as p...",02:29,07:11,34.5


In [25]:
final_result.shape

(20790, 8)

In [26]:
len(final_result['CATEGORY'].unique())

55

In [27]:
final_result['CATEGORY'].unique()

array(['Personal care', 'Sleep', 'Eating',
       'Other and/or unspecified personal care',
       'Employment, related activities and travel as part of/during main and second job',
       'Main and second job and related travel',
       'Activities related to employment and unspecified employment',
       'Study', 'School and university except homework', 'Homework',
       'Free time study', 'Household and family care',
       'Food management except dish washing', 'Dish washing',
       'Cleaning dwelling', 'Household upkeep except cleaning dwelling',
       'Laundry', 'Ironing',
       'Handicraft and producing textiles and other care for textiles',
       'Gardening', 'Tending domestic animals', 'Caring for pets',
       'Walking the dog', 'Construction and repairs ',
       'Shopping and services',
       'Childcare, except teaching, reading and talking',
       'Teaching, reading and talking with child',
       'Household management and help family member',
       'Leisure, socia

In [371]:
final = final_result

In [372]:
final.loc[final['CATEGORY'] == 'Other and/or unspecified personal care', 'CATEGORY'] = 'Personal care'
final.loc[final['CATEGORY'] == 'Resting', 'CATEGORY'] = 'Sleep'
final.loc[final['CATEGORY'] == 'School and university except homework', 'CATEGORY'] = 'Work / Study'
final.loc[final['CATEGORY'] == 'Study', 'CATEGORY'] = 'Work / Study'
final.loc[final['CATEGORY'] == 'Homework', 'CATEGORY'] = 'Work / Study'
final.loc[final['CATEGORY'] == 'Free time study', 'CATEGORY'] = 'Work / Study'
final.loc[final['CATEGORY'] == 'Employment, related activities and travel as part of/during main and second job', 'CATEGORY'] = 'Work / Study'    
final.loc[final['CATEGORY'] == 'Organisational work', 'CATEGORY'] = 'Work / Study'
final.loc[final['CATEGORY'] == 'Main and second job and related travel', 'CATEGORY'] = 'Work / Study'
final.loc[final['CATEGORY'] == 'Household upkeep except cleaning dwelling', 'CATEGORY'] = 'Household and family care'
final.loc[final['CATEGORY'] == 'Household management and help family member', 'CATEGORY'] = 'Household and family care'

final.loc[final['CATEGORY'] == 'Informal help to other households', 'CATEGORY'] = 'Household and family care'
final.loc[final['CATEGORY'] == 'Gardening', 'CATEGORY'] = 'Home maintenance'
final.loc[final['CATEGORY'] == 'Construction and repairs ', 'CATEGORY'] = 'Home maintenance'
final.loc[final['CATEGORY'] == 'Tending domestic animals', 'CATEGORY'] = 'Pet care'
final.loc[final['CATEGORY'] == 'Caring for pets', 'CATEGORY'] = 'Pet care'
final.loc[final['CATEGORY'] == 'Walking the dog', 'CATEGORY'] = 'Pet care'
final.loc[final['CATEGORY'] == 'Shopping and services', 'CATEGORY'] = 'Shopping'
final.loc[final['CATEGORY'] == 'Childcare, except teaching, reading and talking', 'CATEGORY'] = 'Child care'
final.loc[final['CATEGORY'] == 'Teaching, reading and talking with child', 'CATEGORY'] = 'Child care'
final.loc[final['CATEGORY'] == 'Unspecified leisure ', 'CATEGORY'] = 'Leisure, social and associative life'

final.loc[final['CATEGORY'] == 'Participatory activities', 'CATEGORY'] = 'Leisure, social and associative life'
final.loc[final['CATEGORY'] == 'Visiting and feasts', 'CATEGORY'] = 'Leisure, social and associative life'
final.loc[final['CATEGORY'] == 'Other social life', 'CATEGORY'] = 'Leisure, social and associative life'
final.loc[final['CATEGORY'] == 'Entertainment and culture', 'CATEGORY'] = 'Leisure, social and associative life'
final.loc[final['CATEGORY'] == 'Computer games', 'CATEGORY'] = 'Leisure, social and associative life'
final.loc[final['CATEGORY'] == 'Computing', 'CATEGORY'] = 'Leisure, social and associative life'
final.loc[final['CATEGORY'] == 'Hobbies and games except computing and computer games', 'CATEGORY'] = 'Leisure, social and associative life'
final.loc[final['CATEGORY'] == 'Reading books', 'CATEGORY'] = 'Leisure, social and associative life'
final.loc[final['CATEGORY'] == 'Reading, except books', 'CATEGORY'] = 'Leisure, social and associative life'
final.loc[final['CATEGORY'] == 'TV and video', 'CATEGORY'] = 'Leisure, social and associative life'

final.loc[final['CATEGORY'] == 'Radio and music', 'CATEGORY'] = 'Leisure, social and associative life'
final.loc[final['CATEGORY'] == 'Walking and hiking', 'CATEGORY'] = 'Sport'
final.loc[final['CATEGORY'] == 'Sports and outdoor activities except walking and hiking', 'CATEGORY'] = 'Sport'
final.loc[final['CATEGORY'] == 'Travel except travel related to jobs', 'CATEGORY'] = 'Travel'
final.loc[final['CATEGORY'] == 'Travel related to shopping and services', 'CATEGORY'] = 'Travel'
final.loc[final['CATEGORY'] == 'Unspecified travel', 'CATEGORY'] = 'Travel'
final.loc[final['CATEGORY'] == 'Travel related to study', 'CATEGORY'] = 'Transportations'
final.loc[final['CATEGORY'] == 'Travel to/from work', 'CATEGORY'] = 'Transportations'
final.loc[final['CATEGORY'] == 'Transporting a child', 'CATEGORY'] = 'Transportations'
final.loc[final['CATEGORY'] == 'Travel related to leisure, social and associative life', 'CATEGORY'] = 'Transportations'

final.loc[final['CATEGORY'] == 'Travel related to other household purposes', 'CATEGORY'] = 'Transportations'
final.loc[final['CATEGORY'] == 'Unspecified time use', 'CATEGORY'] = 'Unspecified'
final.loc[final['CATEGORY'] == 'Activities related to employment and unspecified employment', 'CATEGORY'] = 'Work / Study'
final.loc[final['CATEGORY'] == 'Food management except dish washing', 'CATEGORY'] = 'Household and family care'
final.loc[final['CATEGORY'] == 'Dish washing', 'CATEGORY'] = 'Household and family care'
final.loc[final['CATEGORY'] == 'Cleaning dwelling', 'CATEGORY'] = 'Household and family care'
final.loc[final['CATEGORY'] == 'Laundry', 'CATEGORY'] = 'Household and family care'
final.loc[final['CATEGORY'] == 'Ironing', 'CATEGORY'] = 'Household and family care'
final.loc[final['CATEGORY'] == 'Handicraft and producing textiles and other care for textiles', 'CATEGORY'] = 'Household and family care'


In [373]:
len(final['CATEGORY'].unique())

14

In [374]:
final["DAYSWEEK"].unique()

array(['All days of the week', 'Monday to Friday', 'Saturday to Sunday'],
      dtype=object)

In [375]:
final = final[final["Participation time (hh:mm)"] != ":U"]

In [376]:
final[(final["CATEGORY"] == 'Household and family care') & (final["DAYSWEEK"] == 'Monday to Friday')]

,COUNTRY,YEAR,DAYSWEEK,SEX,CATEGORY,Time spent (hh:mm),Participation time (hh:mm),Participation rate (%)
176,Belgium,2000,Monday to Friday,Total,Household and family care,03:23,03:36,93.8
177,Belgium,2000,Monday to Friday,Total,Household and family care,00:40,00:57,70.7
178,Belgium,2000,Monday to Friday,Total,Household and family care,00:15,00:31,49.9
179,Belgium,2000,Monday to Friday,Total,Household and family care,00:18,00:53,34.3
180,Belgium,2000,Monday to Friday,Total,Household and family care,00:22,00:49,46.0
181,Belgium,2000,Monday to Friday,Total,Household and family care,00:05,00:28,18.0
182,Belgium,2000,Monday to Friday,Total,Household and family care,00:11,01:01,18.3
183,Belgium,2000,Monday to Friday,Total,Household and family care,00:03,01:07,5.1
192,Belgium,2000,Monday to Friday,Total,Household and family care,00:10,00:37,25.5
231,Belgium,2000,Monday to Friday,Males,Household and family care,02:26,02:41,90.5


In [377]:
final['CATEGORY'].unique()

array(['Personal care', 'Sleep', 'Eating', 'Work / Study',
       'Household and family care', 'Home maintenance', 'Pet care',
       'Shopping', 'Child care', 'Leisure, social and associative life',
       'Sport', 'Travel', 'Transportations', 'Unspecified'], dtype=object)

In [378]:
final.head()

,COUNTRY,YEAR,DAYSWEEK,SEX,CATEGORY,Time spent (hh:mm),Participation time (hh:mm),Participation rate (%)
0,Belgium,2000,All days of the week,Total,Personal care,10:58,10:59,100.0
1,Belgium,2000,All days of the week,Total,Sleep,08:25,08:25,100.0
2,Belgium,2000,All days of the week,Total,Eating,01:49,01:50,99.5
3,Belgium,2000,All days of the week,Total,Personal care,00:44,00:45,97.6
4,Belgium,2000,All days of the week,Total,Work / Study,02:29,07:11,34.5


In [379]:
final.shape

(20427, 8)

In [380]:
final.head()

,COUNTRY,YEAR,DAYSWEEK,SEX,CATEGORY,Time spent (hh:mm),Participation time (hh:mm),Participation rate (%)
0,Belgium,2000,All days of the week,Total,Personal care,10:58,10:59,100.0
1,Belgium,2000,All days of the week,Total,Sleep,08:25,08:25,100.0
2,Belgium,2000,All days of the week,Total,Eating,01:49,01:50,99.5
3,Belgium,2000,All days of the week,Total,Personal care,00:44,00:45,97.6
4,Belgium,2000,All days of the week,Total,Work / Study,02:29,07:11,34.5


In [381]:
final["Time spent (hh:mm)"] = final["Time spent (hh:mm)"].fillna('00:00')
final["Participation time (hh:mm)"] = final["Participation time (hh:mm)"].fillna('00:00')
final["Participation rate (%)"] = final["Participation rate (%)"].fillna('0')

/Users/Alex/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Users/Alex/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/Alex/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentati

In [382]:
final.loc[final['Time spent (hh:mm)'] == ':', 'Time spent (hh:mm)'] = '00:00'
final.loc[final['Participation time (hh:mm)'] == ':', 'Participation time (hh:mm)'] = '00:00'

/Users/Alex/anaconda/lib/python3.6/site-packages/pandas/core/indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [383]:
final.isnull().sum()

COUNTRY                       0
YEAR                          0
DAYSWEEK                      0
SEX                           0
CATEGORY                      0
Time spent (hh:mm)            0
Participation time (hh:mm)    0
Participation rate (%)        0
dtype: int64

In [384]:
def get_sec(time_str):
    h, m = time_str.split(':')
    return int(h) * 3600 + int(m) * 60

def get_sec_reversed(time):
    h, m = str(datetime.timedelta(seconds = time)).split(':')[:2]
    return h + ":" + m

In [385]:
final['Time spent (hh:mm)'] = final['Time spent (hh:mm)'].apply(get_sec)
final["Participation time (hh:mm)"] = final["Participation time (hh:mm)"].apply(get_sec)

/Users/Alex/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Users/Alex/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [386]:
final[final['Participation rate (%)'] == ":"] = 0

/Users/Alex/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Users/Alex/anaconda/lib/python3.6/site-packages/pandas/core/indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [387]:
final.head()

,COUNTRY,YEAR,DAYSWEEK,SEX,CATEGORY,Time spent (hh:mm),Participation time (hh:mm),Participation rate (%)
0,Belgium,2000,All days of the week,Total,Personal care,39480,39540,100.0
1,Belgium,2000,All days of the week,Total,Sleep,30300,30300,100.0
2,Belgium,2000,All days of the week,Total,Eating,6540,6600,99.5
3,Belgium,2000,All days of the week,Total,Personal care,2640,2700,97.6
4,Belgium,2000,All days of the week,Total,Work / Study,8940,25860,34.5


In [388]:
gb = final.groupby(['COUNTRY', 'YEAR', 'DAYSWEEK', 'SEX', 'CATEGORY'])['Time spent (hh:mm)'].sum()

In [389]:
gb = pd.DataFrame(gb.reset_index(name = 'SUM Time spent (hh:mm)'))
gb["SUM Time spent (hh:mm)"] = gb["SUM Time spent (hh:mm)"] / 2
gb = gb.drop(0)

In [392]:
gb["SUM Time spent (hh:mm)"] = gb["SUM Time spent (hh:mm)"].apply(get_sec_reversed)

In [393]:
gb2 = final.groupby(['COUNTRY', 'YEAR', 'DAYSWEEK', 'SEX', 'CATEGORY'])['Participation time (hh:mm)'].sum()

In [394]:
gb2 = pd.DataFrame(gb2.reset_index(name = 'SUM Participation time (hh:mm)'))
gb2["SUM Participation time (hh:mm)"] = gb2["SUM Participation time (hh:mm)"] / 2
gb2 = gb2.drop(0)

In [397]:
gb2["SUM Participation time (hh:mm)"] = gb2["SUM Participation time (hh:mm)"].apply(get_sec_reversed)

In [398]:
final['Participation rate (%)'] = final['Participation rate (%)'].apply(pd.to_numeric)

/Users/Alex/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [399]:
gb3 = final.groupby(['COUNTRY', 'YEAR', 'DAYSWEEK', 'SEX', 'CATEGORY'])['Participation rate (%)'].mean()
gb3 = pd.DataFrame(gb3.reset_index(name = 'MEAN Participation rate (%)'))
gb3 = gb3.drop(0)

In [ ]:
gb["SUM Participation time (hh:mm)"] = gb2["SUM Participation time (hh:mm)"]
gb['MEAN Participation rate (%)'] = gb3['MEAN Participation rate (%)']

In [456]:
gb4 = final.groupby(['YEAR', 'DAYSWEEK', 'SEX', 'CATEGORY'])['Time spent (hh:mm)'].mean()
gb4 = pd.DataFrame(gb4.reset_index(name = 'SUM Time spent (hh:mm)'))
gb4 = gb4.drop(0)

In [457]:
gb4["SUM Time spent (hh:mm)"] = gb4["SUM Time spent (hh:mm)"].apply(get_sec_reversed)

In [458]:
gb4['COUNTRY'] = 'Europe'

In [465]:
gb5 = final.groupby(['YEAR', 'DAYSWEEK', 'SEX', 'CATEGORY'])['Participation time (hh:mm)'].mean()
gb5 = pd.DataFrame(gb5.reset_index(name = 'SUM Participation time (hh:mm)'))
gb5 = gb5.drop(0)

In [466]:
gb5["SUM Participation time (hh:mm)"] = gb5["SUM Participation time (hh:mm)"].apply(get_sec_reversed)

In [467]:
gb5['COUNTRY'] = 'Europe'

In [474]:
gb6 = final.groupby(['YEAR', 'DAYSWEEK', 'SEX', 'CATEGORY'])['Participation rate (%)'].mean()
gb6 = pd.DataFrame(gb6.reset_index(name = 'MEAN Participation rate (%)'))
gb6 = gb6.drop(0)

In [475]:
gb6['COUNTRY'] = 'Europe'

In [476]:
gb4["SUM Participation time (hh:mm)"] = gb5["SUM Participation time (hh:mm)"]
gb4['MEAN Participation rate (%)'] = gb6['MEAN Participation rate (%)']

In [412]:
#gb.to_csv('dataDONEComma.csv', sep = ",")

In [426]:
data = pd.read_csv("dataDONEComma.csv", sep = ",", index_col = 0)

In [483]:
country = 'France'
daysweek = 'All days of the week'
sex = "Males"
year = 2010

In [485]:
test = data[(data['YEAR'] == year) & (data['COUNTRY'] == country) & (data['DAYSWEEK'] == daysweek) & (data['SEX'] == sex)]

In [486]:
test.head()

,COUNTRY,YEAR,DAYSWEEK,SEX,CATEGORY,SUM Time spent (hh:mm),SUM Participation time (hh:mm),MEAN Participation rate (%)
1167,France,2010,All days of the week,Males,Child care,0:06,0:59,11.550000
1168,France,2010,All days of the week,Males,Eating,1:07,1:08,99.200000
1169,France,2010,All days of the week,Males,Home maintenance,0:12,1:09,8.500000
1170,France,2010,All days of the week,Males,Household and family care,1:43,6:17,20.900000
1171,France,2010,All days of the week,Males,"Leisure, social and associative life",4:30,12:34,22.546154


In [491]:
category = ['Eating', 'Child care']
test[test['CATEGORY'].isin(category)]

['Males', 'Males']

In [478]:
gb = pd.concat([gb, gb4])

In [493]:
gb.to_csv('dataDONEComma.csv', sep = ",")